In [1]:
import os
import sys
import numpy as np
import h5py
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork
from data.CleanSoundsDataset import CleanSoundsDataset, TrainCleanSoundsDataset, PsychophysicsCleanSoundsDataset

# Specify Network to train
TODO: This should be converted to a script that accepts arguments for which network to train

In [2]:
from pbranchednetwork_conv3 import PBranchedNetwork_Conv3SeparateHP
PNetClass = PBranchedNetwork_Conv3SeparateHP
pnet_name = 'conv3'

# Parameters

In [3]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
BATCH_SIZE = 20
NUM_WORKERS = 2
PIN_MEMORY = True
NUM_EPOCHS = 50

lr = 1E-4
engram_dir = '/mnt/smb/locker/issa-locker/users/Erica/'
checkpoints_dir = f'{engram_dir}hcnn/checkpoints/'
tensorboard_dir = f'{engram_dir}hcnn/tensorboard/'
train_datafile = f'{engram_dir}seed_542_word_clean_random_order.hdf5'

Device: cuda:0


In [4]:
!nvidia-smi

Thu Jul  7 13:15:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:1D:00.0 Off |                  N/A |
| 27%   28C    P8    21W / 250W |      3MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load network and optimizer

In [5]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

/share/issa/users/es3773/hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


<All keys matched successfully>

In [6]:
pnet = PNetClass(net, build_graph=True)

In [7]:
pnet.eval()

PBranchedNetwork_Conv3SeparateHP(
  (backbone): BranchedNetwork(
    (speech_branch): Sequential(
      (conv1): ConvLayer(
        (block): Sequential(
          (0): Conv2d(1, 96, kernel_size=(6, 14), stride=(3, 3), padding=(2, 6))
          (1): ReLU()
        )
      )
      (rnorm1): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool1): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv2): ConvLayer(
        (block): Sequential(
          (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 2))
          (1): ReLU()
        )
      )
      (rnorm2): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool2): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv3): ConvLayer(
        (block): Sequential(
          (0): Conv2d(256, 512, kernel_siz

In [8]:
pnet.to(DEVICE)
optimizer = torch.optim.Adam(
    [{'params':getattr(pnet,f"pcoder{x+1}").pmodule.parameters(), 'lr':lr} for x in range(pnet.number_of_pcoders)],
    weight_decay=5e-4)

# Set up TrainSoundsDataset

In [ ]:
train_dataset = TrainCleanSoundsDataset(train_datafile)

# Set up PsychophysicsSoundsDataset

In [ ]:
f_in = h5py.File(f"{engram_dir}PsychophysicsWord2017W_not_resampled.hdf5", 'r')

In [ ]:
f_metadata = np.load(f"{engram_dir}PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META.npy", 'r')

In [ ]:
f_key = np.load(f"{engram_dir}PsychophysicsWord2017W_999c6fc475be1e82e114ab9865aa5459e4fd329d.__META_key.npy", 'r')

In [ ]:
def getPsychophysics2017WCleanCochleagrams():
    
    cochleagrams_clean = []
   
    cochleagrams = []
    for batch_ii in range(0,15300,100):
        hdf5_path = '/mnt/smb/locker/issa-locker/users/Erica/cgrams_for_noise_robustness_analysis/PsychophysicsWord2017W_clean/batch_'+str(batch_ii)+'_to_'+str(batch_ii+100)+'.hdf5'
        with h5py.File(hdf5_path, 'r') as f_in:
            cochleagrams += list(f_in['data'])

    return cochleagrams
clean_in = getPsychophysics2017WCleanCochleagrams()
clean_in = np.array(clean_in)

In [ ]:
labels = []
for word in f_metadata['word']:
    idx = np.argwhere(f_key == word)
    if len(idx) == 0:
        labels.append(-1)
    else:
        labels.append(idx.item())
labels = np.array(labels)
labels += 1

In [ ]:
orig_dset = []
for _orig_dset in f_metadata['orig_dset']:
    _orig_dset = str(_orig_dset, 'utf-8')
    _orig_dset = 'WSJ' if 'WSJ' in _orig_dset else 'Timit'
    orig_dset.append(_orig_dset)
orig_dset = np.array(orig_dset)

In [ ]:
psycho_dataset = PsychophysicsCleanSoundsDataset(
    clean_in, labels, orig_dset, exclude_timit=True
    )

In [ ]:
del orig_dset
del clean_in
del labels

# Merge Datasets

In [ ]:
full_dataset = CleanSoundsDataset(train_dataset, psycho_dataset)
del train_dataset
del psycho_dataset
n_train = int(len(full_dataset)*0.9)
train_dataset = Subset(full_dataset, np.arange(n_train))
eval_dataset = Subset(full_dataset, np.arange(n_train, len(full_dataset)))
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )
eval_loader = DataLoader(
    eval_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )

torch.Size([55188, 1, 164, 400])


# Set up checkpoints and tensorboards

In [ ]:
checkpoint_path = os.path.join(checkpoints_dir, f"{pnet_name}")
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, pnet_name + '-{epoch}-{type}.pth')

# summarywriter
from torch.utils.tensorboard import SummaryWriter
tensorboard_path = os.path.join(tensorboard_dir, f"{pnet_name}")
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
sumwriter = SummaryWriter(tensorboard_path, filename_suffix=f'')

# Train

In [ ]:
loss_function = torch.nn.MSELoss()
for epoch in range(1, NUM_EPOCHS+1):
    train_pcoders(pnet, optimizer, loss_function, epoch, train_loader, DEVICE, sumwriter)
    eval_pcoders(pnet, loss_function, epoch, eval_loader, DEVICE, sumwriter)

    # save checkpoints every 5 epochs
    if epoch % 5 == 0:
        torch.save(pnet.state_dict(), checkpoint_path.format(epoch=epoch, type='regular'))

/home/es3773/.local/lib/python3.7/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Training Epoch: 1 [20/49669]	Loss: 3450874.7500
Training Epoch: 1 [40/49669]	Loss: 3454960.2500
Training Epoch: 1 [60/49669]	Loss: 2546383.5000
Training Epoch: 1 [80/49669]	Loss: 3489636.7500
Training Epoch: 1 [100/49669]	Loss: 3712058.7500
Training Epoch: 1 [120/49669]	Loss: 3743765.5000
Training Epoch: 1 [140/49669]	Loss: 3907733.2500
Training Epoch: 1 [160/49669]	Loss: 4056351.5000
Training Epoch: 1 [180/49669]	Loss: 4787789.5000
Training Epoch: 1 [200/49669]	Loss: 4600431.5000
Training Epoch: 1 [220/49669]	Loss: 4117565.2500
Training Epoch: 1 [240/49669]	Loss: 4583157.0000
Training Epoch: 1 [260/49669]	Loss: 5482876.0000
Training Epoch: 1 [280/49669]	Loss: 4907575.0000
Training Epoch: 1 [300/49669]	Loss: 4693817.5000
Training Epoch: 1 [320/49669]	Loss: 5633357.0000
Training Epoch: 1 [340/49669]	Loss: 5303913.0000
Training Epoch: 1 [360/49669]	Loss: 6537260.0000
Training Epoch: 1 [380/49669]	Loss: 5464949.0000
Training Epoch: 1 [400/49669]	Loss: 5855531.0000
Training Epoch: 1 [420/4

Training Epoch: 1 [3280/49669]	Loss: 13178119.0000
Training Epoch: 1 [3300/49669]	Loss: 15081459.0000
Training Epoch: 1 [3320/49669]	Loss: 13454031.0000
Training Epoch: 1 [3340/49669]	Loss: 15307393.0000
Training Epoch: 1 [3360/49669]	Loss: 14293281.0000
Training Epoch: 1 [3380/49669]	Loss: 16566142.0000
Training Epoch: 1 [3400/49669]	Loss: 14625781.0000
Training Epoch: 1 [3420/49669]	Loss: 14826871.0000
Training Epoch: 1 [3440/49669]	Loss: 15117072.0000
Training Epoch: 1 [3460/49669]	Loss: 13698702.0000
Training Epoch: 1 [3480/49669]	Loss: 17190434.0000
Training Epoch: 1 [3500/49669]	Loss: 17750048.0000
Training Epoch: 1 [3520/49669]	Loss: 15230950.0000
Training Epoch: 1 [3540/49669]	Loss: 16791548.0000
Training Epoch: 1 [3560/49669]	Loss: 13841657.0000
Training Epoch: 1 [3580/49669]	Loss: 16858142.0000
Training Epoch: 1 [3600/49669]	Loss: 16879770.0000
Training Epoch: 1 [3620/49669]	Loss: 17158398.0000
Training Epoch: 1 [3640/49669]	Loss: 14611072.0000
Training Epoch: 1 [3660/49669]	

Training Epoch: 1 [6520/49669]	Loss: 8535856.0000
Training Epoch: 1 [6540/49669]	Loss: 7545051.0000
Training Epoch: 1 [6560/49669]	Loss: 7431613.0000
Training Epoch: 1 [6580/49669]	Loss: 8774845.0000
Training Epoch: 1 [6600/49669]	Loss: 8799614.0000
Training Epoch: 1 [6620/49669]	Loss: 6962742.5000
Training Epoch: 1 [6640/49669]	Loss: 7842856.0000
Training Epoch: 1 [6660/49669]	Loss: 9005071.0000
Training Epoch: 1 [6680/49669]	Loss: 9142933.0000
Training Epoch: 1 [6700/49669]	Loss: 7665729.5000
Training Epoch: 1 [6720/49669]	Loss: 9364698.0000
Training Epoch: 1 [6740/49669]	Loss: 8431398.0000
Training Epoch: 1 [6760/49669]	Loss: 7798402.5000
Training Epoch: 1 [6780/49669]	Loss: 7182206.5000
Training Epoch: 1 [6800/49669]	Loss: 7002081.0000
Training Epoch: 1 [6820/49669]	Loss: 6808742.5000
Training Epoch: 1 [6840/49669]	Loss: 7695544.5000
Training Epoch: 1 [6860/49669]	Loss: 9095296.0000
Training Epoch: 1 [6880/49669]	Loss: 8565974.0000
Training Epoch: 1 [6900/49669]	Loss: 6822889.0000


Training Epoch: 1 [9800/49669]	Loss: 5001495.0000
Training Epoch: 1 [9820/49669]	Loss: 4819827.5000
Training Epoch: 1 [9840/49669]	Loss: 4676181.5000
Training Epoch: 1 [9860/49669]	Loss: 3773834.5000
Training Epoch: 1 [9880/49669]	Loss: 4217626.5000
Training Epoch: 1 [9900/49669]	Loss: 5049567.0000
Training Epoch: 1 [9920/49669]	Loss: 4411650.0000
Training Epoch: 1 [9940/49669]	Loss: 4836163.0000
Training Epoch: 1 [9960/49669]	Loss: 4381456.5000
Training Epoch: 1 [9980/49669]	Loss: 4213858.5000
Training Epoch: 1 [10000/49669]	Loss: 4450923.0000
Training Epoch: 1 [10020/49669]	Loss: 4699947.5000
Training Epoch: 1 [10040/49669]	Loss: 4312351.5000
Training Epoch: 1 [10060/49669]	Loss: 4153216.7500
Training Epoch: 1 [10080/49669]	Loss: 4610011.5000
Training Epoch: 1 [10100/49669]	Loss: 4249742.0000
Training Epoch: 1 [10120/49669]	Loss: 4499844.5000
Training Epoch: 1 [10140/49669]	Loss: 5074100.5000
Training Epoch: 1 [10160/49669]	Loss: 3476074.0000
Training Epoch: 1 [10180/49669]	Loss: 409

...

Training Epoch: 3 [1060/49669]	Loss: 17101.8652
Training Epoch: 3 [1080/49669]	Loss: 20325.4668
Training Epoch: 3 [1100/49669]	Loss: 18831.2441
Training Epoch: 3 [1120/49669]	Loss: 19243.8340
Training Epoch: 3 [1140/49669]	Loss: 17146.7070
Training Epoch: 3 [1160/49669]	Loss: 19196.7617
Training Epoch: 3 [1180/49669]	Loss: 17737.6582
Training Epoch: 3 [1200/49669]	Loss: 19105.1270
Training Epoch: 3 [1220/49669]	Loss: 18003.9980
Training Epoch: 3 [1240/49669]	Loss: 21508.2168
Training Epoch: 3 [1260/49669]	Loss: 16388.9160
Training Epoch: 3 [1280/49669]	Loss: 17151.4434
Training Epoch: 3 [1300/49669]	Loss: 17889.1465
Training Epoch: 3 [1320/49669]	Loss: 17525.0664
Training Epoch: 3 [1340/49669]	Loss: 17569.7949
Training Epoch: 3 [1360/49669]	Loss: 16844.1973
Training Epoch: 3 [1380/49669]	Loss: 17392.4590
Training Epoch: 3 [1400/49669]	Loss: 18767.2227
Training Epoch: 3 [1420/49669]	Loss: 19819.7637
Training Epoch: 3 [1440/49669]	Loss: 18019.8574
Training Epoch: 3 [1460/49669]	Loss: 191

Training Epoch: 3 [4480/49669]	Loss: 18184.3965
Training Epoch: 3 [4500/49669]	Loss: 18435.3438
Training Epoch: 3 [4520/49669]	Loss: 17972.2031
Training Epoch: 3 [4540/49669]	Loss: 17900.8086
Training Epoch: 3 [4560/49669]	Loss: 17084.8242
Training Epoch: 3 [4580/49669]	Loss: 15658.4834
Training Epoch: 3 [4600/49669]	Loss: 18796.5664
Training Epoch: 3 [4620/49669]	Loss: 16831.2324
Training Epoch: 3 [4640/49669]	Loss: 15426.4600
Training Epoch: 3 [4660/49669]	Loss: 18030.1660
Training Epoch: 3 [4680/49669]	Loss: 16169.9746
Training Epoch: 3 [4700/49669]	Loss: 16590.3633
Training Epoch: 3 [4720/49669]	Loss: 17968.7559
Training Epoch: 3 [4740/49669]	Loss: 19559.3887
Training Epoch: 3 [4760/49669]	Loss: 18864.4980
Training Epoch: 3 [4780/49669]	Loss: 13495.7510
Training Epoch: 3 [4800/49669]	Loss: 15838.0283
Training Epoch: 3 [4820/49669]	Loss: 19089.5801
Training Epoch: 3 [4840/49669]	Loss: 18214.6484
Training Epoch: 3 [4860/49669]	Loss: 20291.7266
Training Epoch: 3 [4880/49669]	Loss: 165

Training Epoch: 3 [7900/49669]	Loss: 18185.2676
Training Epoch: 3 [7920/49669]	Loss: 17641.2363
Training Epoch: 3 [7940/49669]	Loss: 18980.6758
Training Epoch: 3 [7960/49669]	Loss: 19114.7266
Training Epoch: 3 [7980/49669]	Loss: 16302.0391
Training Epoch: 3 [8000/49669]	Loss: 17305.6582
Training Epoch: 3 [8020/49669]	Loss: 19266.2285
Training Epoch: 3 [8040/49669]	Loss: 18378.6250
Training Epoch: 3 [8060/49669]	Loss: 17817.3770
Training Epoch: 3 [8080/49669]	Loss: 15603.9570
Training Epoch: 3 [8100/49669]	Loss: 15089.2051
Training Epoch: 3 [8120/49669]	Loss: 19019.1816
Training Epoch: 3 [8140/49669]	Loss: 16711.1504
Training Epoch: 3 [8160/49669]	Loss: 15209.0381
Training Epoch: 3 [8180/49669]	Loss: 17533.3184
Training Epoch: 3 [8200/49669]	Loss: 15909.0479
Training Epoch: 3 [8220/49669]	Loss: 16122.3047
Training Epoch: 3 [8240/49669]	Loss: 17699.4121
Training Epoch: 3 [8260/49669]	Loss: 17166.3047
Training Epoch: 3 [8280/49669]	Loss: 18259.3379
Training Epoch: 3 [8300/49669]	Loss: 131

Training Epoch: 3 [11300/49669]	Loss: 19099.5039
Training Epoch: 3 [11320/49669]	Loss: 14966.3945
Training Epoch: 3 [11340/49669]	Loss: 17906.9961
Training Epoch: 3 [11360/49669]	Loss: 16601.5234
Training Epoch: 3 [11380/49669]	Loss: 15776.1592
Training Epoch: 3 [11400/49669]	Loss: 15018.8301
Training Epoch: 3 [11420/49669]	Loss: 19045.3672
Training Epoch: 3 [11440/49669]	Loss: 16773.1777
Training Epoch: 3 [11460/49669]	Loss: 16601.1035
Training Epoch: 3 [11480/49669]	Loss: 19084.3066
Training Epoch: 3 [11500/49669]	Loss: 16858.9648
Training Epoch: 3 [11520/49669]	Loss: 17010.9746
Training Epoch: 3 [11540/49669]	Loss: 17362.5703
Training Epoch: 3 [11560/49669]	Loss: 16498.4102
Training Epoch: 3 [11580/49669]	Loss: 16812.2148
Training Epoch: 3 [11600/49669]	Loss: 17312.2188
Training Epoch: 3 [11620/49669]	Loss: 15668.7324
Training Epoch: 3 [11640/49669]	Loss: 12396.1523
Training Epoch: 3 [11660/49669]	Loss: 13803.1729
Training Epoch: 3 [11680/49669]	Loss: 17118.2988
Training Epoch: 3 [1

Training Epoch: 3 [14660/49669]	Loss: 15641.0283
Training Epoch: 3 [14680/49669]	Loss: 18384.7246
Training Epoch: 3 [14700/49669]	Loss: 17269.6074
Training Epoch: 3 [14720/49669]	Loss: 17029.0293
Training Epoch: 3 [14740/49669]	Loss: 17404.9160
Training Epoch: 3 [14760/49669]	Loss: 17837.4512
Training Epoch: 3 [14780/49669]	Loss: 17356.6523
Training Epoch: 3 [14800/49669]	Loss: 17786.2949
Training Epoch: 3 [14820/49669]	Loss: 17289.5879
Training Epoch: 3 [14840/49669]	Loss: 18681.4473
Training Epoch: 3 [14860/49669]	Loss: 13291.5381
Training Epoch: 3 [14880/49669]	Loss: 17235.3809
Training Epoch: 3 [14900/49669]	Loss: 16896.7734
Training Epoch: 3 [14920/49669]	Loss: 17321.2773
Training Epoch: 3 [14940/49669]	Loss: 15567.1611
Training Epoch: 3 [14960/49669]	Loss: 17077.6855
Training Epoch: 3 [14980/49669]	Loss: 15442.9717
Training Epoch: 3 [15000/49669]	Loss: 18396.3496
Training Epoch: 3 [15020/49669]	Loss: 14915.5850
Training Epoch: 3 [15040/49669]	Loss: 15634.2256
Training Epoch: 3 [1